The Knapsack problem is a typical dynamic programming problem that can be solved using integer optimization. In this Jupyter note, we will go over the basic concepts of knapsack problem as well as using Google's OR tool for optimal scheduling. This is our first attempt that delves into the field of **operational research (OR)**.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os 

from ortools.linear_solver import pywraplp

In [2]:
path="C:\\Users\\gao\\GAO_Jupyter_Notebook\\Datasets"
os.chdir(path)

#path="C:\\Users\\pgao\\Documents\\PGZ Documents\\Programming Workshop\\PYTHON\\Open Courses on Python\\Udemy Course on Python\Introduction to Data Science Using Python\\datasets"
#os.chdir(path)

In the **knapsack problem**, we need to pack a set of items, with given values and sizes (such as weights or volumes), into a container or a set of containers (or bins) with a maximum capacity. If the total size of the items exceeds the capacity, we can't pack them all. In that case, the problem is to choose a subset of the items of maximum total value that will fit in the container. A multiple knapsack problem refers to the case where we have to fit more than one bins (containers). 

Here, the concept of items, values, sizes (like weights or volumes) and containers are all allegorical. One can think of OR (operating room) scheduling as exactly the same type of problem. In an OR scheduling problem, we need to pack a set of operations with given values (revenue to the hospital) and sizes (estimated surgery length), into a set of containers (actual operating rooms); each room has some maximum capacity (in terms of how long the room should be open for utilization). A similar situation is that we have a bunch of goods with different weights and values (for selling) to be packed onto one or a set of trucks. The question is how should allocate the space in the truck so that we maximize our values subject to the weight constraints and other constraints etc.

Let's see the code. First, to create such a model, we need to have 3 vectors:

   1. A vector of weights: this is a Python list containing the weights of each item to be allocated to the container. In the OR scheduling problem, this is the estimated duration of each operational procedure.
   2. A vector of values: this is a Python list containing the values of the items. In the OR scheduling problem, this is the potential (estimated) revenue of each operational procedure. If these are unknown, we can set the whole vector to be a generic number (mean value of all operational procedure revenue).
   3. A vector of capacities: this is a Python list cotaining the capacities of the bins. Bins basically the containers. In the OR scheduling problem, this is the room. 

In [12]:
data = {}
data['weights'] = [50, 30, 420, 120, 36, 48, 150, 420, 36, 240, 30, 300, 100, 30, 30, 60]

data['values'] = [7600, 1000, 3000, 2500, 500, 3500, 3000, 1520, 4000, 1000, 2230, 4500, 1000, 2800, 9000, 900] # revenue
assert len(data['weights']) == len(data['values'])
data['num_items'] = len(data['weights'])
data['all_items'] = range(data['num_items'])

data['bin_capacities'] = [360, 360, 360, 360, 360] # in theory it should be less than len(data['weights'])
data['num_bins'] = len(data['bin_capacities'])
data['all_bins'] = range(data['num_bins'])

data

{'weights': [50,
  30,
  420,
  120,
  36,
  48,
  150,
  420,
  36,
  240,
  30,
  300,
  100,
  30,
  30,
  60],
 'values': [7600,
  1000,
  3000,
  2500,
  500,
  3500,
  3000,
  1520,
  4000,
  1000,
  2230,
  4500,
  1000,
  2800,
  9000,
  900],
 'num_items': 16,
 'all_items': range(0, 16),
 'bin_capacities': [360, 360, 360, 360, 360],
 'num_bins': 5,
 'all_bins': range(0, 5)}

Now let's declare the solver (MIP). This is a mixed integer programming problem.

In [13]:
solver = pywraplp.Solver.CreateSolver('SCIP')

Now let's create the choice variable. Each $x_{i,b}$ is a 0-1 binary variable, where $i$ stands for the item and $j$ is the bin. In the solution, $x_{i,b}$ will be 1 if item $i$ is placed in bin $j$, and 0 otherwise:

In [14]:
x = {}
for i in data['all_items']:
    for b in data['all_bins']:
        x[i, b] = solver.BoolVar(f'x_{i}_{b}')
print('a particular instance:', x[14,4])
x

a particular instance: x_14_4


{(0, 0): x_0_0,
 (0, 1): x_0_1,
 (0, 2): x_0_2,
 (0, 3): x_0_3,
 (0, 4): x_0_4,
 (1, 0): x_1_0,
 (1, 1): x_1_1,
 (1, 2): x_1_2,
 (1, 3): x_1_3,
 (1, 4): x_1_4,
 (2, 0): x_2_0,
 (2, 1): x_2_1,
 (2, 2): x_2_2,
 (2, 3): x_2_3,
 (2, 4): x_2_4,
 (3, 0): x_3_0,
 (3, 1): x_3_1,
 (3, 2): x_3_2,
 (3, 3): x_3_3,
 (3, 4): x_3_4,
 (4, 0): x_4_0,
 (4, 1): x_4_1,
 (4, 2): x_4_2,
 (4, 3): x_4_3,
 (4, 4): x_4_4,
 (5, 0): x_5_0,
 (5, 1): x_5_1,
 (5, 2): x_5_2,
 (5, 3): x_5_3,
 (5, 4): x_5_4,
 (6, 0): x_6_0,
 (6, 1): x_6_1,
 (6, 2): x_6_2,
 (6, 3): x_6_3,
 (6, 4): x_6_4,
 (7, 0): x_7_0,
 (7, 1): x_7_1,
 (7, 2): x_7_2,
 (7, 3): x_7_3,
 (7, 4): x_7_4,
 (8, 0): x_8_0,
 (8, 1): x_8_1,
 (8, 2): x_8_2,
 (8, 3): x_8_3,
 (8, 4): x_8_4,
 (9, 0): x_9_0,
 (9, 1): x_9_1,
 (9, 2): x_9_2,
 (9, 3): x_9_3,
 (9, 4): x_9_4,
 (10, 0): x_10_0,
 (10, 1): x_10_1,
 (10, 2): x_10_2,
 (10, 3): x_10_3,
 (10, 4): x_10_4,
 (11, 0): x_11_0,
 (11, 1): x_11_1,
 (11, 2): x_11_2,
 (11, 3): x_11_3,
 (11, 4): x_11_4,
 (12, 0): x_12_0,
 (

Now let's define the constraints of the problem. This is the hardest part. In a multiple knapsack problem, we require at least 2 constraints: 

   1. Each item is assigned to at most one bin.
   2. The amount of items packed in each bin cannot exceed its maximum capacity. 
   
In the OR scheduling problem, these 2 constraints can be translated as the following: each planned operation must be assigned at most one room (doctor cannot shift rooms during a single operation); in addition, the cumulative time of surgeries/operations within each operating room cannot exceed its maximum capacity. The maximum capacity is essentially defined as the maximum amount of minutes that can be used to perform these operations. Suppose we require that on each day, the operating room can function from 9:00am till 3:00pm. This way, the room has a maximum capacity of 6 hours ($6 \times 60=360$ minutes). In reality, if we jam-pack 6 operations into one single room, we cannot implement such a plan because we have to set some time for cleaning and transition. We also have to consider that all the estimates of duration for each operation has some margin of error. So a realistic solution would be to set a utilization rate for each room (or a subset of all rooms). Let $ur$ define the utilization rate where its range is between 0 and 1. So this a hyperparameter that a scheduler can utilize. The higher the number, the more revenue the hospital can generate but it also runs the risk of all delays for subsequent operations. So there is a tradeoff certainly just like in an economic decisions made. So let's revise our maximum capacity. In the example, we will we will set the utilization rate to be 85%:

In [15]:
ur=0.85
data['bin_capacities'] = [360, 360, 360, 360, 360]
data['bin_capacities'] =[j * ur for j in data['bin_capacities']]
data.update({'bin_capacities':data['bin_capacities']})
data

{'weights': [50,
  30,
  420,
  120,
  36,
  48,
  150,
  420,
  36,
  240,
  30,
  300,
  100,
  30,
  30,
  60],
 'values': [7600,
  1000,
  3000,
  2500,
  500,
  3500,
  3000,
  1520,
  4000,
  1000,
  2230,
  4500,
  1000,
  2800,
  9000,
  900],
 'num_items': 16,
 'all_items': range(0, 16),
 'bin_capacities': [306.0, 306.0, 306.0, 306.0, 306.0],
 'num_bins': 5,
 'all_bins': range(0, 5)}

In [16]:
# Each item is assigned to at most one bin:
for i in data['all_items']:
    solver.Add(sum(x[i, b] for b in data['all_bins']) <= 1)

# The amount (weights) packed in each bin cannot exceed its capacity:
for b in data['all_bins']:
    solver.Add(sum(x[i, b] * data['weights'][i]
                   for i in data['all_items']) <= data['bin_capacities'][b])

We now define the objective function, which is to maximize the total value of the packed items. In the OR scheduling problem, this refers to maximizing the potential revenue of the cases to be performed subject to the given constraints of the room utilization defined before:

In [17]:
objective = solver.Objective()

for i in data['all_items']:
    for b in data['all_bins']:
        objective.SetCoefficient(x[i, b], data['values'][i])
        
objective.SetMaximization()

Note that $x_{i,b}$ times the element data['value'][i] adds the value of item $i$ to the objective if the item is placed in bin $j$. If $i$ is not  placed in any bin, its value doesn't contribute to the objective. 

Lastly, let's invoke the solver:

In [18]:
status=solver.Solve()

Now print the solution:

In [20]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Total packed value: ${objective.Value()}", '\n')
    total_weight = 0
    for b in data['all_bins']:
        print(f'Bin {b}:')
        bin_weight = 0
        bin_value = 0
        for i in data['all_items']:
            if x[i, b].solution_value() > 0:
                print(
                    f"Item {i} weight: {data['weights'][i]} (weights) value: ${data['values'][i]}"
                )
                bin_weight += data['weights'][i]
                bin_value += data['values'][i]
        print(f'Packed bin weight: {bin_weight}')
        print(f'Packed bin value: {bin_value}\n')
        total_weight += bin_weight
    print(f'Total packed weight: {total_weight}')
else:
    print('The problem does not have an optimal solution.')

Total packed value: $43530.0 

Bin 0:
Item 0 weight: 50 (weights) value: $7600
Item 4 weight: 36 (weights) value: $500
Item 5 weight: 48 (weights) value: $3500
Item 6 weight: 150 (weights) value: $3000
Packed bin weight: 284
Packed bin value: 14600

Bin 1:
Item 1 weight: 30 (weights) value: $1000
Item 3 weight: 120 (weights) value: $2500
Item 8 weight: 36 (weights) value: $4000
Item 10 weight: 30 (weights) value: $2230
Item 13 weight: 30 (weights) value: $2800
Item 15 weight: 60 (weights) value: $900
Packed bin weight: 306
Packed bin value: 13430

Bin 2:
Item 11 weight: 300 (weights) value: $4500
Packed bin weight: 300
Packed bin value: 4500

Bin 3:
Item 9 weight: 240 (weights) value: $1000
Item 14 weight: 30 (weights) value: $9000
Packed bin weight: 270
Packed bin value: 10000

Bin 4:
Item 12 weight: 100 (weights) value: $1000
Packed bin weight: 100
Packed bin value: 1000

Total packed weight: 1260


#### References:

   - https://developers.google.com/optimization/bin/multiple_knapsack
   - https://developers.google.com/optimization/bin/bin 
   